In [ ]:
import os
import PyPDF2
import json
import pandas as pd
import traceback

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv

In [ ]:
load_dotenv()  # it load the environment variabel

In [ ]:
key=os.getenv("OPENAI_API_KEY")

In [ ]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.7)

In [ ]:
with open("C:/Users/princ/genrativeAi/genaiprojcet/MCQ_genrator_genai/Response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [ ]:
#print(RESPONSE_JSON)

In [ ]:
TEMPLATE= """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [ ]:
quiz_genration_prompt= PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [ ]:
## it connect the prompt and llm model
quiz_chain=LLMChain(llm=llm,prompt=quiz_genration_prompt,output_key="quiz",verbose="True")

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt= PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [ ]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose="True")

In [ ]:
# it connect both the chains
genrate_evaluation_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","RESPONSE_JSON"],output_variables=["quiz","review"],verbose=True)

In [ ]:
PATH="C:/Users/princ/genrativeAi/genaiprojcet/MCQ_genrator_genai/data.txt"

In [ ]:
with open(PATH,"r") as file:
    TEXT=file.read()

In [ ]:
#print(TEXT)

In [ ]:
# Generate mcq
genrate_evaluation_chain

In [ ]:
TEXT
NUMBER=5
SUBJECT="Biology"
TONE="SIMPLE"
RESPONSE_JSON=RESPONSE_JSON

In [ ]:
#RESPONSE_JSON

In [ ]:
import json
json.dumps(RESPONSE_JSON)

In [ ]:
with get_openai_callback() as cb:
   response= genrate_evaluation_chain(
      {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "RESPONSE_JSON":json.dumps(RESPONSE_JSON) 
      }                      
    )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
#response

In [ ]:
QUIZ=response.get("quiz")

In [ ]:
#QUIZ

In [ ]:
QUIZ=json.loads(QUIZ)

In [ ]:
quiz_table_data=[]
for key, value in QUIZ.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
#quiz_table_data

In [ ]:
df=pd.DataFrame(quiz_table_data)

In [ ]:
df.to_csv("genai.csv",index=False)